In [1]:
from mmseg.apis import MMSegInferencer

from mmseg.registry import DATASETS
from mmseg.datasets import BaseSegDataset

old_classes = ['Outside billboards', 'Billboard', 'Field player', 'Goalkeeper', 'Referee', 'Assistant referee', 'Other human', 'Ball', 'Goal post', 'Goal net', 'Net post', 'Cross-bar', 'Corner flag', 'Assistant flag', 'Microphone', 'Camera', 'Other object', 'Don\'t care']
old_palette = [[0, 0, 0], [255, 255, 255], [255, 0, 0], [0, 255, 0], [0, 0, 255], [255, 255, 0], [255, 0, 255], [0, 255, 255], [128, 0, 0], [0, 128, 0], [0, 0, 128], [64, 64, 64], [128, 128, 0], [128, 0, 128], [0, 128, 128], [255, 128, 0], [192, 192, 192], [128, 128, 128]]

new_classes = ['Outside billboards', 'Billboard', 'Goal net']
new_palette = [old_palette[old_classes.index(c)] for c in new_classes]

# Check if the dataset SoccerNet is already registered
if 'SoccerNet' not in DATASETS:
	@DATASETS.register_module()
	class SoccerNet(BaseSegDataset):
		METAINFO = dict(classes = new_classes, palette = new_palette)
		def __init__(self, **kwargs):
			super().__init__(img_suffix='.png', seg_map_suffix='.png', **kwargs)

In [2]:
inferencer = MMSegInferencer(model='mask2former_swin-t_1xb2-90k_soccernet.py', weights='best_mIoU_iter_46200.pth', classes=new_classes, palette=new_palette)

Loads checkpoint by local backend from path: best_mIoU_iter_46200.pth
01/05 16:49:40 - mmengine - WARNING - Failed to search registry with scope "mmseg" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmseg" is a correct scope, or whether the registry is initialized.


/home/vgaspar/miniconda3/envs/mmseg/lib/python3.11/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [7]:
img_list = ["Stadium_4_Match_2_in_1fps_6662", "Stadium_4_Match_2_in022", "Stadium_4_Match_2_in_1fps_6615", "Stadium_4_Match_2_in_1fps_6451", "Stadium_4_Match_2_in_1fps_6497", "Stadium_4_Match_2_in_1fps_5734", "Stadium_4_Match_2_in_1fps_5374", "Stadium_4_Match_2_in_1fps_5363", "Stadium_4_Match_2_in_1fps_5353", "Stadium_4_Match_2_in_1fps_5356", "Stadium_4_Match_2_in_1fps_5000", "Stadium_4_Match_2_in_1fps_4524", "Stadium_4_Match_2_in_1fps_4289", "Stadium_4_Match_2_in_1fps_2717", "Stadium_4_Match_2_in_1fps_2061", "Stadium_4_Match_2_in_1fps_2058"]

In [8]:
# For each item in the list, add 'Dataset/Images/' and '.png' to get the path to the image
img_list = ['Dataset/Images/' + i + '.png' for i in img_list]

In [9]:
inferencer(img_list, out_dir='out', with_labels=False, opacity=0.5)

Output()

FileNotFoundError: [Errno 2] No such file or directory: 'Dataset/Images/Stadium_4_Match_2_in022.png.png'